# Generación de ejemplos

Genero ejemplos de metadatos a partir del ejemplo de `data.json`.

In [87]:
from __future__ import unicode_literals
import json
from pprint import pprint
from functools import partial
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

In [88]:
with open("examples/data.json", "rb") as f:
    data = json.load(f)
with open("meta_names.json", "rb") as f:
    meta_names = json.load(f)
with open("meta_class_names.json", "rb") as f:
    meta_class_names = json.load(f)

## Exploración del data.json

In [89]:
# campos de metadata de un catalog
print len(data)
data.keys()

13


[u'rights',
 u'publisher',
 u'license',
 u'description',
 u'language',
 u'title',
 u'issued',
 u'superThemeTaxonomy',
 u'modified',
 u'themeTaxonomy',
 u'spatial',
 u'dataset',
 u'homepage']

In [90]:
# campos de metadata de un dataset
print len(data["dataset"][0].keys())
data["dataset"][0].keys()

17


[u'publisher',
 u'landingPage',
 u'description',
 u'superTheme',
 u'title',
 u'issued',
 u'temporal',
 u'modified',
 u'language',
 u'theme',
 u'keyword',
 u'accrualPeriodicity',
 u'spatial',
 u'license',
 u'contactPoint',
 u'identifier',
 u'distribution']

In [91]:
# campos de metadata de un distribucion
print len(data["dataset"][0]["distribution"][0].keys())
data["dataset"][0]["distribution"][0].keys()

12


[u'accessURL',
 u'rights',
 u'description',
 u'license',
 u'title',
 u'byteSize',
 u'format',
 u'mediaType',
 u'modified',
 u'downloadURL',
 u'field',
 u'issued']

In [92]:
# campos de metadata de un field
print len(data["dataset"][0]["distribution"][0]["field"][0].keys())
data["dataset"][0]["distribution"][0]["field"][0].keys()

3


[u'type', u'description', u'title']

In [93]:
# campos de metadata de un theme
print len(data["themeTaxonomy"][0].keys())
data["themeTaxonomy"][0].keys()

3


[u'description', u'id', u'label']

## Separación de ejemplos en JSON

In [94]:
def write_to_json(example_name, example):
    with open("examples/{}.json".format(example_name), "wb") as f:
        f.write(json.dumps(example, f, indent=4, ensure_ascii=False).encode("utf8"))

In [95]:
write_to_json("dataset", data["dataset"][0])
write_to_json("distribution", data["dataset"][0]["distribution"][0])
write_to_json("field", data["dataset"][0]["distribution"][0]["field"][0])
write_to_json("theme", data["themeTaxonomy"][0])
write_to_json("themeTaxonomy", data["themeTaxonomy"])

## Creación de ejemplos en Markdown

### Dataset

Secciones de un documento de metadata a nivel de dataset.

1. Título
2. Descripción
3. Características (metadatos del dataset)
4. Distribuciones
    * Título
    * Resto de los metadatos
    * Campos
        + nombre (tipo): descripción

In [100]:
def metadata_to_md(meta_obj, meta_names, meta_class_names, 
                   ignore_fields=["title", "description"], level=1, meta_class="catalog"):
    """Convierte metadatos a nivel de dataset o distribución en markdown."""

    # se asume que el objeto de metadata tiene un título y una descripción
    metadatos = []
    metadatos.append("{head_level} {class_name}: {title}".format(
        head_level="#" * level,
        class_name = meta_class_names[meta_class], 
        title=meta_obj["title"])
    )
    metadatos.append(meta_obj["description"])
    
    # construyo el md de la metadata a nivel de dataset o distribución
    
    # primero agrego los campos simples, los de nuevas secciones se hacen al final
    new_sections = []
    for meta_field in meta_obj:
        if meta_field not in ignore_fields:
            # parseo los campos según el tipo de valor que tienen
            
            # listas de objetos son nueva sección del documento, al final de todo
            if type(meta_obj[meta_field]) == list and type(meta_obj[meta_field][0]) == dict:
                new_sections.append(meta_field)
            
            # las listas son valores separados por coma
            elif type(meta_obj[meta_field]) == list:
                meta_value = ", ".join(meta_obj[meta_field])
                meta_field_name = meta_names[meta_class][meta_field]
                metadatos.append("* **{}:** {}".format(meta_field_name, meta_value))
            
            # los objetos conforman sublistas
            elif type(meta_obj[meta_field]) == dict:
                for meta_sub_field in meta_obj[meta_field]:
                    meta_value = meta_obj[meta_field][meta_sub_field]
                    meta_sub_field_name = meta_names[meta_class][meta_field][meta_sub_field]
                    metadatos.append("* **{}**: {}".format(meta_sub_field_name, meta_value))
                
            # el resto sigue como viene..
            else:
                meta_value = meta_obj[meta_field]
                meta_field_name = meta_names[meta_class][meta_field]
                metadatos.append("* **{}:** {}".format(meta_field_name, meta_value))
    
    # ahora parseo los campos que configuran nuevas secciones
    for meta_field in new_sections:
        if meta_field not in ignore_fields:
            
            # las listas de objetos "field" se parsean de manera específica
            if type(meta_obj[meta_field]) == list and meta_field == "field":
                metadatos.append("#" * (level + 1) + " " + meta_names[meta_class][meta_field])
                for field in meta_obj[meta_field]:
                    metadatos.append("* **{} ({})**: {}".format(
                        field["title"], field["type"], field["description"]))                    
            
            # las listas de objetos "theme" se parsean de manera específica
            elif type(meta_obj[meta_field]) == list and meta_field == "themeTaxonomy":
                metadatos.append("#" * (level + 1) + " " + meta_names[meta_class][meta_field])
                for field in meta_obj[meta_field]:
                    metadatos.append("* **{} ({})**: {}".format(
                        field["label"], field["id"], field["description"]))                    
            
            # las listas de objetos se parsean como una nueva sección del documento
            else:
                metadatos.append("#" * (level + 1) + " " + meta_names[meta_class][meta_field])
                for sub_meta_obj in meta_obj[meta_field]:
                    metadatos.extend(metadata_to_md(sub_meta_obj, meta_names, meta_class_names, 
                                                    level=level + 2, meta_class=meta_field))
    
    return metadatos

In [101]:
def datajson_to_markdown(datajson, meta_names, meta_class_names):
    """Convierte metadatos de un dataset a markdown."""
    return "\n\n".join(metadata_to_md(datajson, meta_names, meta_class_names))

In [103]:
%pdb
catalogo = datajson_to_markdown(data, meta_names, meta_class_names)
printmd(catalogo)

Automatic pdb calling has been turned ON


# Catálogo: Datos Argentina

Portal de Datos Abiertos del Gobierno de la República Argentina

* **Derechos sobre el catálogo:** Derechos especificados en la licencia.

* **Correo electrónico del autor**: datos@modernizacion.gob.ar

* **Autor**: Ministerio de Modernización

* **Licencia:** Open Data Commons Open Database License 1.0

* **Idioma(s):** spa

* **Fecha de creación o publicación:** 2016-04-14T19:48:05.433640-03:00

* **Taxonomía temática global:** http://datos.gob.ar/superThemeTaxonomy.json

* **Fecha de última actualización/modificación:** 2016-04-19T19:48:05.433640-03:00

* **Cobertura geográfica:** ARG

* **Página web del catálogo:** http://datos.gob.ar

## Taxonomía temática específica

* **Convocatorias (convocatorias)**: Datasets sobre licitaciones en estado de convocatoria.

* **Compras (compras)**: Datasets sobre compras realizadas.

* **Contrataciones (contrataciones)**: Datasets sobre contrataciones.

* **Adjudicaciones (adjudicaciones)**: Datasets sobre licitaciones adjudicadas.

* **Normativa (normativa)**: Datasets sobre normativa para compras y contrataciones.

* **Proveedores (proveedores)**: Datasets sobre proveedores del Estado.

## Datasets

### Dataset: Sistema de contrataciones electrónicas

Datos correspondientes al Sistema de Contrataciones Electrónicas (Argentina Compra)

* **Correo electrónico del autor**: onc@modernizacion.gob.ar

* **Autor**: Ministerio de Modernización. Secretaría de Modernización Administrativa. Oficina Nacional de Contrataciones

* **Página de referencias:** http://datos.gob.ar/dataset/sistema-de-contrataciones-electronicas-argentina-compra

* **Temática(s) globales:** ECON

* **Fecha de publicación:** 2016-04-14T19:48:05.433640-03:00

* **Cobertura temporal:** 2015-01-01/2015-12-31

* **Fecha de última actualización/ modificación:** 2016-04-19T19:48:05.433640-03:00

* **Idioma(s):** spa

* **Temática(s) específicas:** contrataciones, compras, convocatorias

* **Etiqueta(s):** bienes, compras, contrataciones

* **Frecuencia de actualización:** R/P1Y

* **Cobertura geográfica:** ARG

* **Licencia:** Open Data Commons Open Database License 1.0

* **Correo electrónico del área/persona de contacto**: onc-compraselectronicas@modernizacion.gob.ar

* **Área/Persona de contacto**: Ministerio de Modernización. Secretaría de Modernización Administrativa. Oficina Nacional de Contrataciones. Dirección de Compras Electrónicas.

* **Identificador:** 99db6631-d1c9-470b-a73e-c62daa32c420

#### Distribuciones

##### Distribución: Convocatorias abiertas durante el año 2015

Listado de las convocatorias abiertas durante el año 2015 en el sistema de contrataciones electrónicas

* **URL de acceso:** http://datos.gob.ar/dataset/sistema-de-contrataciones-electronicas-argentina-compra/archivo/fa3603b3-0af7-43cc-9da9-90a512217d8a

* **Derechos sobre la distribución:** Derechos especificados en la licencia.

* **Licencia:** Open Data Commons Open Database License 1.0

* **Tamaño:** 5120

* **Formato del archivo:** CSV

* **Tipo de archivo:** text/csv

* **Fecha de última actualización/ modificación:** 2016-04-19T19:48:05.433640-03:00

* **URL de descarga:** http://186.33.211.253/dataset/99db6631-d1c9-470b-a73e-c62daa32c420/resource/4b7447cb-31ff-4352-96c3-589d212e1cc9/download/convocatorias-abiertas-anio-2015.csv

* **Fecha de publicación:** 2016-04-14T19:48:05.433640-03:00

###### Campos de la distribución

* **procedimiento_id (integer)**: Identificador único del procedimiento de contratación

* **organismo_unidad_operativa_contrataciones_id (integer)**: Identificador único del organismo que realiza la convocatoria. Organismo de máximo nivel jerárquico al que pertenece la unidad operativa de contrataciones.

* **unidad_operativa_contrataciones_id (integer)**: Identificador único de la unidad operativa de contrataciones

* **organismo_unidad_operativa_contrataciones_desc (string)**: Organismo que realiza la convocatoria. Organismo de máximo nivel jerárquico al que pertenece la unidad operativa de contrataciones.

* **unidad_operativa_contrataciones_desc (string)**: Unidad operativa de contrataciones.

* **tipo_procedimiento_contratacion (string)**: Tipo de procedimiento al que se adecua la contratación.

* **ejercicio_procedimiento_anio (date)**: Año en el que se inició el proceso de la convocatoria.

* **fecha_publicacion_convocatoria (date)**: Fecha de publicación de la convocatoria en formato AAAA-MM-DD, ISO 8601.

* **modalidad_convocatoria (string)**: Modalidad bajo la cual se realiza la convocatoria.

* **clase_convocatoria (string)**: Clase de la convocatoria.

* **objeto_convocatoria (string)**: Objeto/objetivo de la convocatoria

In [106]:
with open("examples/data.md", "wb") as f:
    f.write(catalogo.encode("utf8"))